# Notebook 3: Modelo Als

In [7]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from scipy import sparse
import implicit

In [2]:
#padroniza valores pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
users = pd.read_csv("customers_1.csv")

prods = pd.read_csv("products_1.csv")

transfers = pd.read_csv("transactions_1.csv")

In [9]:
users.head(1)

,Customer_ID,SKU_Category,SKU,Average_Transaction_Amount,Total_Transactions,Total_Products,Total_Spent,First_Order_Date,Last_Order_Date
0,1,"{'0H2': 1, 'N8U': 1}","{'6OUVC': 1, 'CEBU8': 1}",16.29,1,2.00,16.29,2016-01-22,2016-01-22


In [10]:
prods.head(1)

,SKU_N,SKU,SKU_Category,Units_Sold,Total_Amount,Unit_Amount_Average
0,0,00GVC,1TS,2.00,35.36,17.68


In [11]:
transfers.head(1)

,Transaction_P_ID,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,2016-01-02,2547,1,X52,0EM7L,1.00,3.13


In [13]:
# quantidade de celulas de uma matriz de usuarios x produtos, com a maior parte sendo esparsa
22625 * 5242

118600250

In [14]:
# media de produtos por usuario - > seriam em media 5 celulas ocupadas para o usuario e 5237 nulas
soma = 0
for i in users.SKU.to_list():
    soma += len(eval(i))
print(soma/users.shape[0])

4.713900552486188


In [18]:
# criando base para o modelo
df_transfers = pd.DataFrame(columns=['USER', 'SKU', 'QUANTITY'])
df_transfers

,USER,SKU,QUANTITY


In [19]:
# adicionando itens comprados por usuario e sua quantidade, em linhas separadas
%%time
for a, i in users.iterrows():
    for j, k in eval(i.SKU).items():
        df_transfers.loc[len(df_transfers.index)] = [i.Customer_ID, j, k]

Wall time: 7min 55s


In [21]:
# como o id do produto é uma string e o modelo precisa de um numero, a conversao vai ser feita atraves desse dicionario
dict_sku = pd.Series(prods['SKU_N'].values,index=prods.SKU).to_dict()

In [22]:
# como ficou a base que sera utilizada
df_transfers['SKU_ID'] = df_transfers['SKU'].map(dict_sku)
df_transfers.sample(5)

,USER,SKU,QUANTITY,SKU_ID
104875,21994,VHZUD,1,4617
47348,11651,NSGRM,1,3511
56185,13418,BLH3I,2,1722
79235,17173,Q5U5W,1,3834
4922,1457,EV6K0,1,2205


In [8]:
# matriz esparsa a partir da base
sparse_matrix = sparse.csr_matrix((df_transfers["QUANTITY"].astype(float), (df_transfers["USER"], df_transfers["SKU_ID"])))
sparse_matrix

<22626x5242 sparse matrix of type '<class 'numpy.float64'>'
	with 106652 stored elements in Compressed Sparse Row format>

In [9]:
# 1.1 modelo Alternating Least Squares, que faz uma estimativa baseado na filtragem colaborativa da matriz, os produtos que mais
# ... se assemelham e os produtos mais indicados para determinados clientes
# 1.2 poderiam ser testados outros parametros de regularization, iterations e factors porem nao tem maneira boa de testar depois
model_als = implicit.als.AlternatingLeastSquares(
    regularization=0.01,
    iterations=100
)

In [10]:
# fitando modelo
%%time
model_als.fit(sparse_matrix)

  0%|          | 0/100 [00:00<?, ?it/s]

Wall time: 4.18 s


In [11]:
# recomendaçoes para usuarios
%%time
model_als.recommend_all(user_items=sparse_matrix, N=3)

Wall time: 2.02 s


array([[   2,    1,    0],
       [ 374, 3215, 1229],
       [1155, 4629, 2264],
       ...,
       [3053,  335, 2538],
       [4629,  253, 1699],
       [2368, 2919, 1033]])

In [20]:
# itens similares ao de id 732(6 itens, o primeiro sendo ele mesmo)
model_als.similar_items(732, N=6)

(array([ 732, 1483, 3310, 3897, 3725, 1712]),
 array([1.0000001 , 0.9078132 , 0.90533733, 0.90533316, 0.90533316,
        0.90533316], dtype=float32))

In [68]:
model_als.similar_items(5000, N=6)

(array([5000, 4069, 3270, 1021, 4871, 2020]),
 array([1.0000002 , 0.62251407, 0.62250787, 0.50756353, 0.49514356,
        0.49188343], dtype=float32))

In [79]:
# recomendaçoes para usuario de id 20002
model_als.recommend(20002, sparse_matrix[20002])

(array([1926, 1473, 3178, 4470,  865, 3530, 4849,  308, 2650,  987]),
 array([0.18740414, 0.1532486 , 0.14632306, 0.12700988, 0.10459536,
        0.09826373, 0.0937887 , 0.0922004 , 0.09128136, 0.08556952],
       dtype=float32))